In [13]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchmetrics as metrics
import torch.optim as optim

# Define a function to calculate the accuracy of the model
acc = metrics.Accuracy(task = "multiclass",num_classes = 19).to("cuda:0")# Import necessary libraries
def accuracy(outputs, labels):
    return acc(outputs, labels)


In [14]:
class dataset(Dataset):
    def __init__(self,length = 64):
        super(dataset).__init__()
        self.ls = np.linspace(0,18,19)
        self.length = length
    def __getitem__(self, index):
        # np.random.seed(index)
        mult = self.ls[index%19]/3
        x = np.linspace(0,int(3.14*400-1),int(3.14*400))
        a = np.random.random()*5
        b = np.random.random()*17000
        c = np.random.random()*4
        y = a * np.sin(mult*(x+b)) + c
        return torch.tensor(y).reshape(-1,1).to(torch.float).to("cuda:0"),torch.tensor(self.ls[index%19]).to("cuda:0")
    def __len__(self):
        return self.length
ds = dataset()
def newdl():
    return DataLoader(ds,batch_size=16,shuffle=True)
dl = newdl()

In [15]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to("cuda:0")
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to("cuda:0")
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) 
        
        # Decode the hidden state of the last time step
        out = self.fc(h0)
        return out
    

In [16]:

# example usage
input_dim = 1 # assuming single channel
hidden_dim = 128
num_layers = 16
dropout = 0
num_classes = 19

# Define the model, criterion and optimizer
model = LSTMClassifier(input_dim, hidden_dim, num_layers, dropout, num_classes).to("cuda:0")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 1e-3,amsgrad=True)

# Define some training parameters
num_epochs = 5000


In [17]:
# Begin training
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    for i,(ts,cl) in enumerate(dl):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(ts.to("cuda:0"))
        loss = criterion(outputs, cl.to(torch.long))
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Update running loss and accuracy
        running_loss += loss.item()
        curracc = accuracy(outputs, cl)
        
    # Print statistics for the current epoch
    print(f'Epoch: {epoch:5} Loss: {loss.item():.4f} Acc: {curracc:.4f}')

print('Training complete')


Epoch:     0 Loss: 2.9662 Acc: 0.0000
Epoch:     1 Loss: 2.9304 Acc: 0.1250
Epoch:     2 Loss: 2.9325 Acc: 0.0625
Epoch:     3 Loss: 2.9553 Acc: 0.0000
Epoch:     4 Loss: 2.9609 Acc: 0.0000
Epoch:     5 Loss: 2.9347 Acc: 0.0000
Epoch:     6 Loss: 2.9381 Acc: 0.1250
Epoch:     7 Loss: 2.9574 Acc: 0.0625
Epoch:     8 Loss: 2.9378 Acc: 0.0625
Epoch:     9 Loss: 2.9571 Acc: 0.0000
Epoch:    10 Loss: 2.9460 Acc: 0.0000
Epoch:    11 Loss: 2.8817 Acc: 0.1250
Epoch:    12 Loss: 2.9489 Acc: 0.0625
Epoch:    13 Loss: 2.9280 Acc: 0.0000
Epoch:    14 Loss: 3.0025 Acc: 0.0000
Epoch:    15 Loss: 2.9275 Acc: 0.0000
Epoch:    16 Loss: 2.9393 Acc: 0.0625
Epoch:    17 Loss: 2.9541 Acc: 0.0000
Epoch:    18 Loss: 2.9187 Acc: 0.0625
Epoch:    19 Loss: 2.9369 Acc: 0.0625
Epoch:    20 Loss: 2.9542 Acc: 0.0625
Epoch:    21 Loss: 2.9376 Acc: 0.0625
Epoch:    22 Loss: 2.9992 Acc: 0.0625
Epoch:    23 Loss: 2.9558 Acc: 0.0000
Epoch:    24 Loss: 2.9365 Acc: 0.0625


KeyboardInterrupt: 

In [ ]:
ts,labels = next(iter(newdl()))
out = model(ts)
sig = nn.Sigmoid()
sfmx = nn.Softmax()
for a,b in zip(out, labels):
    print(f"{sfmx(sig(a))} | {b} vs {torch.argmax(sig(a))}")

C:\Users\aashr\AppData\Local\Temp\ipykernel_13140\3742794436.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(f"{sfmx(sig(a))} | {b} vs {torch.argmax(sig(a))}")


tensor([0.0549, 0.0550, 0.0549, 0.0550, 0.0549, 0.0549, 0.0550, 0.0512, 0.0514,
        0.0513, 0.0512, 0.0513, 0.0512, 0.0513, 0.0512, 0.0512, 0.0513, 0.0513,
        0.0512], device='cuda:0', grad_fn=<SoftmaxBackward0>) | 14.0 vs 3
tensor([0.0549, 0.0550, 0.0549, 0.0550, 0.0549, 0.0549, 0.0550, 0.0512, 0.0514,
        0.0513, 0.0512, 0.0513, 0.0512, 0.0513, 0.0512, 0.0512, 0.0513, 0.0513,
        0.0512], device='cuda:0', grad_fn=<SoftmaxBackward0>) | 12.0 vs 3
tensor([0.0549, 0.0550, 0.0549, 0.0550, 0.0549, 0.0549, 0.0550, 0.0512, 0.0514,
        0.0513, 0.0512, 0.0513, 0.0512, 0.0513, 0.0512, 0.0512, 0.0513, 0.0513,
        0.0512], device='cuda:0', grad_fn=<SoftmaxBackward0>) | 1.0 vs 3
tensor([0.0549, 0.0550, 0.0549, 0.0550, 0.0549, 0.0549, 0.0550, 0.0512, 0.0514,
        0.0513, 0.0512, 0.0513, 0.0512, 0.0513, 0.0512, 0.0512, 0.0513, 0.0513,
        0.0512], device='cuda:0', grad_fn=<SoftmaxBackward0>) | 2.0 vs 3
tensor([0.0549, 0.0550, 0.0549, 0.0550, 0.0549, 0.0549, 0.0550, 0.